# Azure Extractor 결과 확인

이미지 한 장을 Azure OCR(prebuilt-read 또는 prebuilt-layout)으로 분석하고,
결과물(`text`, `pages`, `tables`)을 출력해서 구조를 확인합니다.

In [53]:
import sys
import json
from pathlib import Path

if str(Path.cwd()) not in sys.path:
    sys.path.insert(0, str(Path.cwd()))

from modules.core.extractors.azure_extractor import get_azure_extractor

# 표 구조(ケース/バラ 구분)가 필요하면 model_id="prebuilt-layout"
extractor = get_azure_extractor(model_id="prebuilt-layout", enable_cache=False)

image_path = Path("debug2/③ 株式会社大物-(1)/page_1_original_image.png")
result = extractor.extract_from_image_raw(image_path=image_path)

In [54]:
if result is None:
    print("결과 없음 (API 키/엔드포인트 또는 이미지 경로 확인)")
else:
    print("=" * 60)
    print("1. result['text'] (전체 텍스트)")
    print("=" * 60)
    print(result.get("text", "")[:1500])
    print("..." if len(result.get("text", "")) > 1500 else "")

1. result['text'] (전체 텍스트)
2025年3月7日
2025年2月度 割戻請求書
株式会社農心ジャパン 御中
株式会社 大物
ICS
社棵 大 式
〒544-0031
物会
大阪市生野区鶴橋4-1-35
電話
06-6718-1111
FAX
06-6718-1110
適格請求書発行事業者 NO. T2120001010348
振込先銀行:三菱UFJ銀行 鶴橋支店 当座3386 カブシキガイシャダイブツ
対象期間:2025年2月1日~2月28日
8%対象金額
10%対象金額
非課税対象金額
合計請求金額(税込)
(税抜) / 589,050
(税 抜) 8,000
644,973
(消費税) 47,123
(消費税) 800
高野



In [3]:
if result:
    print("=" * 60)
    print("2. result['pages'] (페이지별 words + boundingBox)")
    print("=" * 60)
    pages = result.get("pages", [])
    for i, p in enumerate(pages):
        words = p.get("words", [])[:15]  # 처음 15개만
        print(f"\n[페이지 {i+1}] words 수: {len(p.get('words', []))}")
        for w in words:
            bbox = w.get("boundingBox", "")
            print(f"  - {repr(w.get('text'))}  boundingBox={bbox}")
        if len(p.get("words", [])) > 15:
            print(f"  ... 외 {len(p['words'])-15}개")

2. result['pages'] (페이지별 words + boundingBox)

[페이지 1] words 수: 380
  - '作'  boundingBox=[2716, 95, 2740, 96, 2740, 133, 2716, 133]
  - '成'  boundingBox=[2755, 96, 2779, 96, 2778, 133, 2755, 133]
  - '日'  boundingBox=[2794, 96, 2820, 96, 2819, 133, 2793, 133]
  - '2025'  boundingBox=[2864, 95, 2942, 95, 2942, 133, 2864, 132]
  - '年'  boundingBox=[2943, 95, 2973, 95, 2973, 133, 2943, 133]
  - '3'  boundingBox=[3033, 96, 3052, 96, 3053, 132, 3033, 132]
  - '月'  boundingBox=[3053, 96, 3082, 96, 3082, 132, 3053, 132]
  - '7'  boundingBox=[3145, 96, 3163, 96, 3163, 133, 3146, 133]
  - '日'  boundingBox=[3163, 96, 3190, 95, 3191, 133, 3164, 133]
  - '2'  boundingBox=[3325, 95, 3344, 94, 3344, 131, 3326, 130]
  - '頁'  boundingBox=[3356, 94, 3389, 95, 3389, 131, 3356, 131]
  - '請'  boundingBox=[2976, 154, 3000, 154, 3000, 190, 2976, 190]
  - '求'  boundingBox=[3014, 154, 3037, 154, 3037, 190, 3014, 190]
  - '書'  boundingBox=[3056, 155, 3078, 154, 3078, 190, 3056, 190]
  - 'NO'  boundingBox=[3088

In [4]:
if result:
    print("=" * 60)
    print("3. result['tables'] (prebuilt-layout일 때만 존재)")
    print("=" * 60)
    tables = result.get("tables", [])
    print(f"표 개수: {len(tables)}")
    for ti, tbl in enumerate(tables):
        cells = tbl.get("cells", [])
        print(f"\n[표 {ti+1}] 셀 개수: {len(cells)}")
        for c in cells[:25]:  # 처음 25개 셀
            print(f"  row={c.get('rowIndex')} col={c.get('columnIndex')}  content={repr(c.get('content',''))}")
        if len(cells) > 25:
            print(f"  ... 외 {len(cells)-25}개 셀")

3. result['tables'] (prebuilt-layout일 때만 존재)
표 개수: 1

[표 1] 셀 개수: 1020
  row=0 col=0  content='請求No.'
  row=0 col=1  content='倉'
  row=0 col=2  content='商品コード'
  row=0 col=3  content='商 品 名'
  row=0 col=4  content='ケース内 入数'
  row=0 col=5  content=''
  row=0 col=6  content='金 額'
  row=0 col=7  content='数 量'
  row=0 col=12  content='条 件'
  row=0 col=25  content='配送 支店'
  row=0 col=26  content='請求金額'
  row=0 col=29  content='摘 要'
  row=1 col=7  content='ケー'
  row=1 col=8  content='ス'
  row=1 col=9  content=''
  row=1 col=10  content='バラ'
  row=1 col=11  content=''
  row=1 col=12  content='単価'
  row=1 col=14  content=''
  row=1 col=15  content='条件%'
  row=1 col=17  content=''
  row=1 col=18  content='販売価格'
  row=1 col=20  content=''
  row=1 col=21  content='区'
  row=1 col=22  content='条件'
  ... 외 995개 셀


### 4. 표 구조 복원 → DataFrame

`result["tables"]`의 셀(rowIndex, columnIndex, content, rowSpan, columnSpan)을 2차원 그리드로 채운 뒤 pandas DataFrame으로 만듭니다.

---
## RAG 실험: 지정 경로 벡터 DB + Azure OCR → LLM

1. **지정한 데이터 경로**에서 `Page*_answer.json` + `Page*.png`를 스캔  
2. 각 페이지 이미지를 **Azure OCR**으로 읽어 `ocr_text`로 쓰고, **벡터 DB(FAISS)** 구축  
3. **쿼리 이미지 경로**를 Azure OCR로 읽은 뒤, 이 텍스트로 RAG 검색  
4. 검색된 예제 + 쿼리 텍스트로 **프롬프트** 구성 → **LLM** 호출 → **answer.json** 형식 결과 출력

In [18]:
# ========== 설정 (경로 등) ==========
from pathlib import Path

# 벡터 DB에 넣을 데이터 폴더 (Page*_answer.json + Page*.png 가 있는 폴더)
DATA_PATH = Path("img/mail/03/조건청구서③ 旭食品?東支店25.02?理分①")
# 쿼리할 이미지 (여기서 OCR 읽고 RAG 검색 → LLM으로 answer.json 생성)
QUERY_IMAGE_PATH = Path("debug2/旭食品?東支店25.03?理分②/page_2_original_image.png")

TOP_K = 3                    # RAG 검색 예제 개수
SIMILARITY_THRESHOLD = 0.3    # 유사도 임계값 (낮을수록 더 많이 후보 포함)
AZURE_MODEL_ID = "prebuilt-layout"  # Azure OCR 모델

# 경로 보정 (프로젝트 루트 기준)
if not DATA_PATH.is_absolute():
    DATA_PATH = Path.cwd() / DATA_PATH
if not QUERY_IMAGE_PATH.is_absolute():
    QUERY_IMAGE_PATH = Path.cwd() / QUERY_IMAGE_PATH

print("DATA_PATH:", DATA_PATH, "→ 존재:", DATA_PATH.exists())
print("QUERY_IMAGE_PATH:", QUERY_IMAGE_PATH, "→ 존재:", QUERY_IMAGE_PATH.exists())

DATA_PATH: /Users/nongshim/Desktop/Python/react_rebate/img/mail/03/조건청구서③ 旭食品?東支店25.02?理分① → 존재: True
QUERY_IMAGE_PATH: /Users/nongshim/Desktop/Python/react_rebate/debug2/旭食品?東支店25.03?理分②/page_2_original_image.png → 존재: True


In [47]:
# ========== 1. 데이터 경로 스캔 + Azure OCR (표 복원) → 벡터 DB 구축 ==========
import re
import faiss
import json
import os
import pandas as pd

from modules.core.extractors.azure_extractor import get_azure_extractor
from modules.utils.text_normalizer import normalize_ocr_text

# SentenceTransformer (RAG와 동일 모델)
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

def preprocess_ocr_text(ocr_text: str) -> str:
    text = re.sub(r"\s+", " ", ocr_text or "")
    text = re.sub(r"(\d+),(\d+)", r"\1\2", text)
    text = re.sub(r"\n+", " ", text)
    return text.strip()

def _azure_table_to_dataframe(table: dict) -> pd.DataFrame:
    """Azure 표 한 개를 DataFrame으로 복원 (rowSpan/columnSpan 반영)."""
    cells = table.get("cells") or []
    if not cells:
        return pd.DataFrame()
    max_r = max(c.get("rowIndex", 0) + (c.get("rowSpan") or 1) - 1 for c in cells)
    max_c = max(c.get("columnIndex", 0) + (c.get("columnSpan") or 1) - 1 for c in cells)
    grid = [["" for _ in range(max_c + 1)] for _ in range(max_r + 1)]
    for cell in cells:
        r, c = cell.get("rowIndex", 0), cell.get("columnIndex", 0)
        rs, cs = cell.get("rowSpan") or 1, cell.get("columnSpan") or 1
        content = (cell.get("content") or "").strip()
        for rr in range(r, min(r + rs, len(grid))):
            for cc in range(c, min(c + cs, len(grid[0]))):
                grid[rr][cc] = content
    return pd.DataFrame(grid)

def raw_to_table_restored_text(raw: dict) -> str:
    """Azure raw 결과를 표 복원 텍스트로 변환. 테이블이 있으면 TSV(헤더+행)로 열 구분 명확히. 없으면 result['text'] 사용."""
    tables = raw.get("tables") or []
    if not tables:
        text = (raw.get("text") or "").strip()
        if not text and raw.get("pages"):
            text = "\n".join(
                " ".join(w.get("text", "") for w in p.get("words") or [])
                for p in raw["pages"]
            )
        return normalize_ocr_text(text or "", use_fullwidth=True)
    parts = []
    for ti, tbl in enumerate(tables):
        df = _azure_table_to_dataframe(tbl)
        if df.empty:
            continue
        # 첫 행을 헤더로 사용 → LLM이 열 이름(ケース/バラ 등)으로 매핑 가능. (0,1,2... 쓰면 위치로만 추측해 스왑 발생)
        header = "\t".join(str(df.iloc[0, j]) for j in range(len(df.columns)))
        rows = ["\t".join(str(df.iloc[i, j]) for j in range(len(df.columns))) for i in range(1, len(df))]
        parts.append(header + "\n" + "\n".join(rows))
    return normalize_ocr_text("\n\n".join(parts) or "", use_fullwidth=True)

# DATA_PATH에서 Page*_answer.json 수집
answer_files = sorted(DATA_PATH.glob("Page*_answer*.json"))
index_pages = []  # [{ "page_key", "ocr_text", "answer_json" }, ...]
azure_extractor = get_azure_extractor(model_id=AZURE_MODEL_ID, enable_cache=True)

for ans_path in answer_files:
    m = re.search(r"Page(\d+)_answer", ans_path.name)
    if not m:
        continue
    page_num = int(m.group(1))
    img_path = DATA_PATH / f"Page{page_num}.png"
    if not img_path.exists():
        img_path = DATA_PATH / f"page_{page_num}.png"
    if not img_path.exists():
        print(f"  ⚠ 이미지 없음: {ans_path.name} → Page{page_num}.png 스킵")
        continue
    with open(ans_path, "r", encoding="utf-8") as f:
        answer_json = json.load(f)
    raw = azure_extractor.extract_from_image_raw(image_path=img_path)
    if not raw:
        print(f"  ⚠ Azure OCR 실패: {img_path.name} 스킵")
        continue
    # 표 복원 텍스트 사용 (케이스/바라 열 구분 가능)
    ocr_text = raw_to_table_restored_text(raw)
    page_key = f"{DATA_PATH.name}_Page{page_num}"
    index_pages.append({
        "page_key": page_key,
        "ocr_text": ocr_text,
        "answer_json": answer_json,
    })
    print(f"  ✅ {img_path.name} → 표복원텍스트 {len(ocr_text)}자")

if not index_pages:
    raise RuntimeError("인덱싱할 페이지가 없습니다. DATA_PATH와 answer/json·png 구성을 확인하세요.")

# FAISS 인덱스 + 메타데이터
processed = [preprocess_ocr_text(p["ocr_text"]) for p in index_pages]
embeddings = embedding_model.encode(processed, convert_to_numpy=True).astype("float32")
dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dim)
faiss_index.add(embeddings)
index_metadata = {p["page_key"]: p for p in index_pages}
id_to_idx = {p["page_key"]: i for i, p in enumerate(index_pages)}
idx_to_id = {i: p["page_key"] for i, p in enumerate(index_pages)}

print(f"\n✅ 벡터 DB 구축 완료: {len(index_pages)}페이지, dim={dim}")

  ✅ Page1.png → 표복원텍스트 896자
  ✅ Page2.png → 표복원텍스트 2677자
  ✅ Page3.png → 표복원텍스트 2469자
  ✅ Page4.png → 표복원텍스트 2534자
  ✅ Page5.png → 표복원텍스트 2812자
  ✅ Page6.png → 표복원텍스트 2804자
  ✅ Page7.png → 표복원텍스트 1430자
  ✅ Page8.png → 표복원텍스트 617자
  ✅ Page9.png → 표복원텍스트 1150자

✅ 벡터 DB 구축 완료: 9페이지, dim=384


In [48]:
# ========== 2. 쿼리 이미지 Azure OCR (표 복원) + RAG 검색 ==========
query_raw = azure_extractor.extract_from_image_raw(image_path=QUERY_IMAGE_PATH)
if not query_raw:
    raise RuntimeError("쿼리 이미지 Azure OCR 실패. 경로와 API 키를 확인하세요.")

# 표 복원 텍스트 사용 → ケース/バラ 열 구분된 문자열로 프롬프트에 전달
query_text_for_prompt = raw_to_table_restored_text(query_raw)
query_processed = preprocess_ocr_text(query_text_for_prompt)
query_embedding = embedding_model.encode([query_processed], convert_to_numpy=True).astype("float32")

k = min(TOP_K * 2, faiss_index.ntotal)
distances, indices = faiss_index.search(query_embedding, k)
rag_results = []
for dist, idx in zip(distances[0], indices[0]):
    if idx == -1:
        continue
    doc_id = idx_to_id.get(idx)
    if not doc_id:
        continue
    similarity = max(0.0, 1.0 - (dist / 100.0))
    if similarity < SIMILARITY_THRESHOLD:
        continue
    data = index_metadata.get(doc_id, {})
    rag_results.append({
        "id": doc_id,
        "similarity": float(similarity),
        "ocr_text": data.get("ocr_text", ""),
        "answer_json": data.get("answer_json", {}),
    })
rag_results = sorted(rag_results, key=lambda x: x["similarity"], reverse=True)[:TOP_K]

print(f"RAG 검색 결과: {len(rag_results)}개")
for r in rag_results:
    print(f"  - {r['id']}  similarity={r['similarity']:.3f}")
# query_text_for_prompt = 표 복원 텍스트 (이미 위에서 설정됨)

RAG 검색 결과: 3개
  - 조건청구서③ 旭食品?東支店25.02?理分①_Page7  similarity=0.999
  - 조건청구서③ 旭食品?東支店25.02?理分①_Page4  similarity=0.999
  - 조건청구서③ 旭食品?東支店25.02?理分①_Page2  similarity=0.998


In [49]:
# 복원된 텍스트를 pandas DataFrame으로 확인하고 엑셀 파일로 내보내기 (TSV 블록별 파싱)
import io
import pandas as pd

blocks = [b.strip() for b in query_text_for_prompt.split("\n\n") if b.strip()]
query_restored_dfs = []
for block in blocks:
    if "\t" in block:
        query_restored_dfs.append(pd.read_csv(io.StringIO(block), sep="\t", dtype=str))
    else:
        query_restored_dfs.append(pd.DataFrame({"text": [ln for ln in block.split("\n") if ln]}))

# 엑셀 파일로 내보내기 (여러 표는 시트별로 저장)
with pd.ExcelWriter("restored_query_tables.xlsx") as writer:
    for i, df in enumerate(query_restored_dfs):
        sheet_name = f'table_{i+1}'
        df.to_excel(writer, index=False, sheet_name=sheet_name)
print(f"✅ 복원된 표를 restored_query_tables.xlsx로 저장 완료 (시트 수: {len(query_restored_dfs)})")

✅ 복원된 표를 restored_query_tables.xlsx로 저장 완료 (시트 수: 1)


In [50]:
# ========== 3. 프롬프트 구성 + LLM 호출 → answer.json ==========
from modules.utils.config import load_rag_prompt, rag_config, get_effective_rag_provider

effective_provider, effective_model = get_effective_rag_provider()
model_name = effective_model or (rag_config.gemini_extractor_model if effective_provider == "gemini" else rag_config.openai_model)

if rag_results:
    example = rag_results[1]
    example_ocr = example["ocr_text"]
    example_answer_str = json.dumps(example["answer_json"], ensure_ascii=False, indent=2)
else:
    example_ocr = ""
    example_answer_str = "{}"

prompt_template = load_rag_prompt()
prompt = prompt_template.format(
    example_ocr=example_ocr,
    example_answer_str=example_answer_str,
    ocr_text=query_text_for_prompt,
)
print(f"프롬프트 길이: {len(prompt)} 문자, LLM: {effective_provider} / {model_name}")

프롬프트 길이: 17885 문자, LLM: gpt / gpt-5.2-2025-12-11


In [51]:
# LLM 호출
import os
if effective_provider == "gemini":
    import google.generativeai as genai
    api_key = os.getenv("GEMINI_API_KEY")
    if not api_key:
        raise ValueError("GEMINI_API_KEY가 필요합니다.")
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(prompt, generation_config={"max_output_tokens": 8000})
    result_text = ""
    if response.candidates and response.candidates[0].content:
        for part in response.candidates[0].content.parts:
            if hasattr(part, "text") and part.text:
                result_text += part.text
else:
    from openai import OpenAI
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OPENAI_API_KEY가 필요합니다.")
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
        timeout=120,
        max_completion_tokens=8000,
    )
    result_text = (response.choices[0].message.content or "").strip()

# JSON 추출 (마크다운 코드블록 제거)
result_text = result_text.strip()
if result_text.startswith("```"):
    result_text = result_text.split("```", 1)[1]
    if result_text.startswith("json"):
        result_text = result_text[4:].strip()
result_text = result_text.rstrip("`").strip()
answer_result = json.loads(result_text)
print("✅ LLM 응답 파싱 완료 (answer.json 형식)")

✅ LLM 응답 파싱 완료 (answer.json 형식)


In [52]:
# 최종 결과 (answer.json)
print(json.dumps(answer_result, ensure_ascii=False, indent=2))
# Jupyter에서 보기 좋게
display(answer_result) if "display" in dir() else None

{
  "page_role": "detail",
  "document_number": null,
  "区_mapping": null,
  "items": [
    {
      "請求番号": "60000610911",
      "得意先": null,
      "商品コード": "05534-00167-00",
      "商品名": "農心 イワシカルグクス 98",
      "内入数": "20×2",
      "金額": null,
      "ケース": null,
      "バラ": "20",
      "単価": "7800",
      "条件％": null,
      "販売価格": null,
      "区": null,
      "条件": null,
      "配送支店": null,
      "請求金額": "1560",
      "備考": "阿部貴弘",
      "消費税率": "8%",
      "タイプ": "販売個別 販促金"
    },
    {
      "請求番号": "60000615719",
      "得意先": "株式会社ベルク",
      "商品コード": "05534-00054-00",
      "商品名": "農心 本場韓国コムタンラーメン 75G",
      "内入数": "12×2",
      "金額": null,
      "ケース": null,
      "バラ": "360",
      "単価": "5600",
      "条件％": null,
      "販売価格": null,
      "区": null,
      "条件": null,
      "配送支店": null,
      "請求金額": "20160",
      "備考": "西口佳宏",
      "消費税率": "8%",
      "タイプ": null
    },
    {
      "請求番号": "60000615719",
      "得意先": "株式会社ベルク",
      "商品コード": "05534-00073-00",
      "商品名":

In [6]:
import pandas as pd


def azure_table_to_dataframe(table: dict) -> pd.DataFrame:
    """
    Azure Layout의 표 한 개(table dict with 'cells')를 2차원 그리드로 복원한 뒤 DataFrame으로 반환.
    rowSpan/columnSpan이 있으면 해당 구간을 모두 같은 content로 채움.
    """
    cells = table.get("cells") or []
    if not cells:
        return pd.DataFrame()

    max_r = max(
        c.get("rowIndex", 0) + (c.get("rowSpan") or 1) - 1
        for c in cells
    )
    max_c = max(
        c.get("columnIndex", 0) + (c.get("columnSpan") or 1) - 1
        for c in cells
    )
    grid = [["" for _ in range(max_c + 1)] for _ in range(max_r + 1)]

    for cell in cells:
        r = cell.get("rowIndex", 0)
        c = cell.get("columnIndex", 0)
        rs = cell.get("rowSpan") or 1
        cs = cell.get("columnSpan") or 1
        content = (cell.get("content") or "").strip()
        for rr in range(r, min(r + rs, len(grid))):
            for cc in range(c, min(c + cs, len(grid[0]))):
                grid[rr][cc] = content

    return pd.DataFrame(grid)


def azure_tables_to_dataframes(result: dict) -> list[pd.DataFrame]:
    """result['tables'] 전체를 DataFrame 리스트로 변환."""
    tables = result.get("tables") or []
    return [azure_table_to_dataframe(t) for t in tables]

In [17]:
# 표를 DataFrame으로 복원해서 엑셀 파일로 내보내기
if result:
    dfs = azure_tables_to_dataframes(result)
    # 여러 표를 여러 시트로 저장
    with pd.ExcelWriter("azure_tables_output.xlsx", engine="xlsxwriter") as writer:
        for i, df in enumerate(dfs):
            sheet_name = f"Table_{i+1}"
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    print("✅ 표를 'azure_tables_output.xlsx'로 저장했습니다.")

✅ 표를 'azure_tables_output.xlsx'로 저장했습니다.


In [5]:
# 전체 결과를 JSON으로 보기 (구조 확인용)
if result:
    # boundingBox 등은 길어서 일부만 표시
    def shorten(obj, max_len=80):
        if isinstance(obj, list) and len(obj) > 4:
            return f"<list len={len(obj)}>"
        s = json.dumps(obj, ensure_ascii=False)
        return s if len(s) <= max_len else s[:max_len] + "..."
    preview = {
        "text": (result.get("text") or "")[:500] + ("..." if len(result.get("text") or "") > 500 else ""),
        "pages": [
            {"words_count": len(p.get("words", [])), "words_sample": [w.get("text") for w in p.get("words", [])[:5]]}
            for p in result.get("pages", [])
        ],
        "tables": [
            {"cells_count": len(t.get("cells", [])), "cells_sample": [{"row": c.get("rowIndex"), "col": c.get("columnIndex"), "content": c.get("content")} for c in t.get("cells", [])[:8]]}
            for t in result.get("tables", [])
        ],
    }
    print(json.dumps(preview, ensure_ascii=False, indent=2))

{
  "text": "作成日 2025年 3月 7日 2頁 請求書NO 0001926042\n条件請求明細書\n株式会社 農心ジャパン 御中\n( 0- 502-05534-51-92-502-2 )\n2025年 2月 28日締\n事業所 502 旭食品株式会社 関東支店\n請求No.\n倉\n商品コード\n商 品 名\nケース内 入数\n金 額\n数 量\n条 件\n配送 支店\n請求金額\n摘 要\nケース\nバラ\n単価\n条件%\n販売価格\n区\n条件\n販売個別 販促金\n05534-51-92-502-2\n6000610911\n阿部貴弘\n倉\n05534- 00167-00\n農心 イワシカルグクス 98\n20× 2\n20\n78 00\n1560\n*\n小 計\n1560\n*\n* * * 税 込 計\n1,685\n税額(軽減 8 %) 125\n6000615719\n株式会社ベルク\n西口佳宏\n倉\n05534- 00054-00\n農心 本場韓国コムタンラーメン 75G\n12× 2\n360\n56 00\n20160\n倉\n05534- 00073-00\n農心 本場韓国コムタンラーメン 3 333G\n12× 1\n504\n126 00\n63504\n良\n05534-...",
  "pages": [
    {
      "words_count": 380,
      "words_sample": [
        "作",
        "成",
        "日",
        "2025",
        "年"
      ]
    }
  ],
  "tables": [
    {
      "cells_count": 1020,
      "cells_sample": [
        {
          "row": 0,
          "col": 0,
          "content": "請求No."
        },
        {
          "row": 0,
          "col": 1,
          "content": "倉"
        },
        {
         